In [2]:
import math	
import numpy	
import time

def batch_gradient_descent(filenamex, filenamey, C, eta, epsilon, n):
	k = 0
	b = 0
	w_0 = [0.0 for i in range(122)]
	y = []
	target = open(filenamey, "r")
	for line in target:
		y.append(int(line.strip()))
	features = open(filenamex, "r")
	x = []
	for line in features:
		feature = line.strip().split(",")
		thisline = []
		for f in feature:
			thisline.append(int(f))
		x.append(thisline)
	while True:
		fk_before = calculate_convergence(w_0, b, x, y, C, n)
		with open('bgd.txt', 'a') as write_file:
			write_file.write(str(fk_before) + ' ' + str(k) + '\n')
		matrix = multiplication(w_0, b, x, y, n)
		for j in range(len(w_0)):
			w_update = w_0
			sigma_l = 0
			for i in range(n):
				if matrix[i] != 0:
					sigma_l += -y[i] * x[i][j]
			gradient_wb = w_0[j] + C * sigma_l
			w_update[j] = w_0[j] - eta * gradient_wb
		w_0 = w_update
		sigma_l_b = 0
		for i in range(n):
			if matrix[i] != 0: 
				sigma_l_b += -y[i]
		b -= eta * C * sigma_l_b
		k += 1
		print k
		fk_after = calculate_convergence(w_0, b, x, y, C, n)
		print math.fabs(fk_after - fk_before) * 100 / fk_before, fk_after
		if math.fabs(fk_after - fk_before) * 100 / fk_before < epsilon:
			with open('bgd.txt', 'a') as write_file:
				write_file.write(str(fk_after) + ' ' + str(k) + '\n')
			break
	return w_0, b

def stochastic_gradient_descent(filenamex, filenamey, C, eta, epsilon, n):
	i = 1 
	k = 0
	b = 0
	cost = 0
	w_0 = [0.0 for m in range(122)]
	y = []
	target = open(filenamey, "r")
	for line in target:
		y.append(int(line.strip()))
	features = open(filenamex, "r")
	x = []
	for line in features:
		feature = line.strip().split(",")
		thisline = []
		for f in feature:
			thisline.append(int(f))
		x.append(thisline)
	for m in range(n):
		x[m].append(y[m])
	numpy.random.shuffle(x)
	for m in range(n):
		y[m] = x[m][-1]
		del x[m][-1] 
	while True:
		fk_before = calculate_convergence(w_0, b, x, y, C, n)
		with open('sgd.txt', 'a') as write_file:
			write_file.write(str(fk_before) + ' ' + str(k) + '\n')
		matrix = multiplication(w_0, b, x, y, n)
		for j in range(len(w_0)):
			w_update = w_0
			l = 0
			if matrix[i - 1] != 0:
				l += -y[i - 1] * x[i - 1][j]
			gradient_wb = w_0[j] + C * l
			w_update[j] = w_0[j] - eta * gradient_wb
		w_0 = w_update
		l_b = 0
		if matrix[i - 1] != 0:
			l_b += -y[i - 1]
		b -= eta * C * l_b
		i = i % n + 1
		k += 1
		print k
		fk_after = calculate_convergence(w_0, b, x, y, C, n)
		cost = 0.5 * cost + 0.5 * math.fabs(fk_after - fk_before) * 100 / fk_before 
		print cost, fk_after
		if cost < epsilon:
			with open('sgd.txt', 'a') as write_file:
				write_file.write(str(fk_after) + ' ' + str(k) + '\n')
			break
	return w_0, b

def minibatch_gradient_descent(filenamex, filenamey, C, eta, epsilon, n):
	k = 0
	b = 0
	l = 0
	cost = 0 
	batch_size = 20
	w_0 = [0.0 for m in range(122)]
	y = []
	target = open(filenamey, "r")
	for line in target:
		y.append(int(line.strip()))
	features = open(filenamex, "r")
	x = []
	for line in features:
		feature = line.strip().split(",")
		thisline = []
		for f in feature:
			thisline.append(int(f))
		x.append(thisline)
	for m in range(n):
		x[m].append(y[m])
	numpy.random.shuffle(x)
	for m in range(n):
		y[m] = x[m][-1]
		del x[m][-1]
	while True:
		fk_before = calculate_convergence(w_0, b, x, y, C, n)
		with open('mbgd.txt', 'a') as write_file:
			write_file.write(str(fk_before) + ' ' + str(k) + '\n')
		matrix = multiplication(w_0, b, x, y, n)
		for j in range(len(w_0)):
			w_update = w_0
			sigma_L = 0
			for i in range(l * batch_size, min(n, (l + 1) * batch_size)):
				if matrix[i] != 0:
					sigma_L += -y[i] * x[i][j]
			gradient_wb = w_0[j] + C * sigma_L
			w_update[j] = w_0[j] - eta * gradient_wb
		w_0 = w_update
		l_b = 0
		for i in range(l * batch_size, min(n, (l + 1) * batch_size)):
			if matrix[i] != 0: 
				l_b += -y[i]
		b -= eta * C * l_b
		l = (l + 1) % ((n + batch_size - 1) / batch_size)
		k += 1 
		print k
		fk_after = calculate_convergence(w_0, b, x, y, C, n)
		cost = 0.5 * cost + 0.5 * math.fabs(fk_after - fk_before) * 100 / fk_before
		print cost, fk_after
		if cost < epsilon:
			with open('mbgd.txt', 'a') as write_file:
				write_file.write(str(fk_after) + ' ' + str(k) + '\n')
			break

def	multiplication(w_0, b, x, y, n): 
	matrix = [0 for i in range(n)] 
	for i in range(n):
		if y[i] * (numpy.dot(x[i], w_0) + b) < 1:
			matrix[i] = 1
	return matrix	

def	calculate_convergence(w_0, b, x, y, C, n):	
	sigma_max = 0	
	for i in range(n):	
		sigma_wx = 0
		for j in range(len(w_0)):
			sigma_wx += w_0[j] * x[i][j]
		if y[i] * (sigma_wx + b) < 1:
			sigma_max += 1 - y[i] * (sigma_wx + b)
	sigma_w = 0
	for j in range(len(w_0)):
		sigma_w += math.pow(w_0[j],2)
	return 0.5 * sigma_w + C * sigma_max

def calculate_error(w_0, b, filenamex, filenamey):
	y = []
	target = open(filenamey, "r")
	for line in target: 
		y.append(int(line.strip())) 
	features = open(filenamex, "r") 
	prediction = []
	for line in features:
		feature = line.strip().split(",")
		sigma_wx = 0
		index = 0
		for x in feature:
			sigma_wx += int(x) * w_0[index]
			index += 1
		if sigma_wx + b > 0:
			prediction.append(1)
		else:
			prediction.append(-1)
	error = 0.0
	for i in range(len(y)):
		if prediction[i] != y[i]:
			error += 1
	return error / len(y)
 
if __name__ == "__main__": 
	C = 100
	eta = 0.0001 
	epsilon = 0.001
	start_mc = time.clock()
	batch_gradient_descent("features.txt", "target.txt", C, 0.0000003, 0.25, 6414)
	stochastic_gradient_descent("features.txt", "target.txt", C, 0.0001, 0.001, 6414)
	minibatch_gradient_descent("features.txt", "target.txt", C, 0.00001, 0.01, 6414)
	values = [1, 10, 50, 100, 200, 300, 400, 500]
# 	for val in values:
# 		w_0, b = stochastic_gradient_descent("features.train.txt", "target.train.txt", val, eta, epsilon, 6000)
# 		error = calculate_error(w_0, b, "features.test.txt", "target.test.txt")
# 		print error
# 		with open('error.txt', 'a') as write_file:
# 			write_file.write(str(error) + ' ' + str(val) + '\n')
	end_mc = time.clock()
	print end_mc - start_mc

1
38.8929186795 391940.819589
2
8.97753546123 356754.193524
3
12.8536860612 310898.129478
4
0.733854208082 308616.590472
5
0.60308355763 306755.374559
6
0.635908841282 304804.690011
7
0.534761599252 303174.711576
8
0.493377027359 301678.917196
9
0.46355047886 300280.483131
10
0.412533999427 299041.724044
11
0.405504045031 297829.097757
12
0.376058680376 296709.085582
13
0.364757931513 295626.815659
14
0.350897230179 294589.469351
15
0.340717237406 293585.75225
16
0.330624298319 292615.086416
17
0.325581581047 291662.385591
18
0.32038048835 290727.956216
19
0.317959504284 289803.559048
20
0.314548093101 288891.987479
21
0.311428772789 287992.294708
22
0.309059235426 287102.227924
23
0.307254436011 286220.093592
24
0.306131126594 285343.884795
25
0.304308458559 284475.559218
26
0.302665182884 283614.550746
27
0.301576618023 282759.235576
28
0.301007813548 281908.108183
29
0.300733445924 281060.316215
30
0.300617406541 280215.399982
31
0.300581255557 279373.125014
32
0.300059228389 278534

202
0.0955787263548 312028.52165
203
0.048752619556 312022.51038
204
1.16897256032 319165.30629
205
1.79333949672 311448.826147
206
0.89765149873 311442.710847
207
1.27934328028 306269.538223
208
0.640064100058 306267.134252
209
0.535205417171 304949.123642
210
0.35582739085 305487.204433
211
0.431737780965 307038.004639
212
0.471304864923 305469.433601
213
0.236121520384 305472.299441
214
0.118529796806 305475.164995
215
0.0597338837141 305478.030262
216
0.648546586682 309257.891048
217
0.325044849272 309262.663243
218
0.915812550879 313921.953456
219
1.79141229565 322294.289752
220
2.4799548905 332506.176218
221
1.24202777819 332519.811185
222
2.87445412869 347506.081645
223
3.60968283193 332407.249818
224
1.80684089992 332420.542678
225
2.60744171566 321091.5092
226
2.53886034857 313159.653137
227
2.23644135624 319216.230863
228
1.11980379938 319226.338023
229
1.8151601948 311212.107847
230
1.73250654342 306077.565885
231
0.867069523172 306082.56261
232
1.07871131313 310032.108455
2

453
0.705288024649 305600.17804
454
1.74111950982 314086.545225
455
2.35973798542 304731.928312
456
1.18085300747 304725.931097
457
0.591410439456 304719.934483
458
0.29668875612 304713.94042
459
0.149326843038 304707.953004
460
0.0756458075534 304701.966188
461
0.0388047966791 304695.982494
462
0.0203833137495 304690.004874
463
0.0111724934347 304684.027852
464
1.11324432061 297934.313383
465
0.557148502854 297931.177073
466
0.859432533429 301392.292907
467
1.40915813074 295488.368323
468
1.74979411763 301665.34613
469
1.89018251833 295539.817339
470
0.945434631396 295537.787735
471
0.473060655952 295535.758335
472
0.74051657237 292556.839196
473
0.7720198615 294907.601128
474
0.386353039253 294905.577422
475
0.19351949039 294903.554542
476
0.0971026840132 294901.531865
477
0.0488942488821 294899.509389
478
0.551295723519 291792.161522
479
0.371040243506 292348.856507
480
0.186356860701 292353.7489
481
0.648977578053 295603.54819
482
0.325727776846 295610.873174
483
0.164103574187 295

704
0.452475215329 284269.630945
705
0.227383297426 284276.144641
706
0.722154519497 287735.574222
707
0.362341754341 287742.851024
708
0.774144614209 284330.37195
709
0.388203530974 284336.804777
710
0.755206247343 281145.951666
711
0.988691540817 284582.052358
712
0.495509408419 284588.675368
713
0.248918294208 284595.298259
714
0.384174121907 283117.025044
715
0.539584242129 285084.672407
716
0.753575660757 282326.286969
717
0.377791054083 282331.951697
718
0.618191663861 284756.03202
719
0.808312839714 287599.133105
720
0.992804906493 284213.237216
721
1.0737687308 280931.33444
722
0.537926154267 280937.187863
723
0.901982245572 284493.960364
724
0.995186609724 281397.553778
725
0.498633825834 281403.409779
726
0.667311721428 279050.906491
727
0.334393139323 279055.021257
728
0.179482602533 278986.451673
729
0.54762891841 281541.340505
730
0.273820536705 281541.374726
731
0.488548297536 279561.361643
732
0.244575326429 279563.045596
733
0.621911904757 282356.575088
734
0.9007101556

954
0.860471667845 275808.391052
955
0.431487844493 275815.297352
956
0.216995776271 275822.202962
957
0.964369516967 280543.570924
958
0.483459352197 280550.722506
959
1.17110162836 285765.44196
960
1.54828574287 280263.114512
961
0.775425795587 280270.305638
962
1.15932758958 275945.091928
963
1.40231596363 271404.955361
964
1.07070704449 269399.006424
965
0.536365967596 269404.461459
966
0.455428137603 270413.355056
967
0.884257914334 273964.119536
968
1.15951874838 270033.338286
969
1.07980599469 272733.92345
970
0.5410999407 272740.452392
971
0.271746892215 272746.981372
972
0.137074345813 272753.532207
973
0.0697379236843 272760.082387
974
0.0360695637149 272766.631913
975
0.669670461358 276321.521303
976
0.336087990187 276328.444592
977
1.17355590286 281885.475421
978
0.588026505575 281892.514407
979
1.29256143871 276262.849229
980
0.6474951692 276269.559377
981
1.06975946235 272147.551922
982
0.536085616986 272154.115499
983
0.269248544637 272160.67842
984
0.135829858818 272167

...

1686
0.354658681086 254157.32148
1687
0.714993107552 251424.297822
1688
0.650562823892 249950.618198
1689
0.32610013927 249954.711026
1690
0.163868701686 249958.803445
1691
0.362540077253 251361.600877
1692
0.478228725134 249868.720661
1693
0.415873183662 250752.050671
1694
0.32130168842 250183.520062
1695
0.161385812212 250187.197599
1696
0.0814287014198 250190.879331
1697
0.0414507488077 250194.564132
1698
0.0214617844834 250198.249048
1699
0.0114716958364 250201.956004
1700
0.00647678819906 250205.663698
1701
0.00397924931241 250209.371021
1702
0.109954118735 250749.645584
1703
0.055498101998 250752.258609
1704
0.341143300321 252323.944927
1705
0.492130383361 250701.205565
1706
0.283570907325 250889.260127
1707
0.415202855935 252261.209923
1708
0.508636355751 250742.42122
1709
0.28930264545 250566.979418
1710
0.145190378108 250569.680808
1711
0.0731341847242 250572.381927
1712
0.0371060283277 250575.082777
1713
0.352778577799 252250.054742
1714
0.176736444295 252251.806141
1715
0.79

1929
0.0178802897036 261516.341249
1930
1.17475970673 255418.723922
1931
0.588166913657 255422.74452
1932
0.294870426031 255426.764717
1933
0.148222091142 255430.784512
1934
1.12643176798 260806.686665
1935
1.64183310225 255180.475008
1936
1.69888886481 250699.647167
1937
0.850224022668 250703.556027
1938
0.425892118715 250707.467541
1939
0.698611138412 248272.2703
1940
0.866164083225 250838.703034
1941
1.44302257411 255905.3465
1942
0.722317490556 255909.472736
1943
0.361964855166 255913.598559
1944
1.11846184535 251115.323931
1945
0.560043477682 251119.404832
1946
0.825029587548 248382.1639
1947
0.976650371576 251184.588211
1948
1.02846618147 248471.086339
1949
0.705670339298 247519.753917
1950
0.353420960063 247522.653811
1951
0.17729620501 247525.553415
1952
0.0892337620549 247528.452729
1953
0.0452024751578 247531.351753
1954
0.0231867662975 247534.250488
1955
0.112085246443 248031.754049
1956
0.163348763573 247499.447445
1957
0.0822719436211 247502.405369
1958
0.148547238093 2480

2175
1.47190726158 258303.950569
2176
0.73664844521 258307.540035
2177
0.369018957895 258311.129143
2178
1.09421590648 253611.383253
2179
0.547809917487 253614.943775
2180
0.953252198784 250169.091534
2181
0.47730135316 250172.470086
2182
0.594350821053 248392.742411
2183
0.297723894222 248395.467198
2184
0.149410369948 248398.191713
2185
0.438479949608 250205.411588
2186
0.571322680764 248443.551621
2187
0.286209454181 248446.275127
2188
0.207318315812 248765.347666
2189
0.10406341984 248767.358993
2190
0.0524359281642 248769.370119
2191
0.295746356671 250110.378288
2192
0.66900639813 252717.194823
2193
0.334615882586 252717.764364
2194
0.574802928428 250658.139922
2195
0.585654236654 249162.950218
2196
0.293185007703 249164.733674
2197
0.146951333492 249166.521828
2198
0.0738359093821 249168.317036
2199
0.0372781587133 249170.112064
2200
0.0189992447692 249171.906913
2201
0.461167398775 251422.765336
2202
0.556206077812 249785.387759
2203
0.344325189534 249454.561248
2204
0.190174992

208
0.0160578332864 288440.997126
209
0.0446972706979 288229.463993
210
0.0699566150621 288503.904443
211
0.125983784223 289029.01315
212
0.189565199186 288297.345989
213
0.154849730897 287951.002099
214
0.0934660179659 288043.383417
215
0.0741330095424 287885.53564
216
0.0665453766466 287715.804824
217
0.0384684089734 287745.702643
218
0.0228474821625 287724.90854
219
0.0261846252177 287809.850021
220
0.0753073642482 288167.972115
221
0.117047914214 287710.394617
222
0.102610879159 287456.709303
223
0.0845659406812 287265.490219
224
0.049494058319 287306.920153
225
0.0333362911113 287356.275241
226
0.0257471099022 287304.097293
227
0.0907173407764 287751.394066
228
0.135803725772 288271.907881
229
0.174355093982 287658.158361
230
0.11452903394 287815.515928
231
0.219386235813 288748.738851
232
0.251945840953 287927.232963
233
0.19617573527 287522.966919
234
0.131502534863 287715.116604
235
0.0899110859079 287854.139504
236
0.082920249698 288072.705464
237
0.159983032638 288755.569758


455
0.0585375139921 267118.724523
456
0.0498989909381 267228.938958
457
0.07161731327 266979.51913
458
0.057262741209 266864.963106
459
0.0527042115433 266736.47915
460
0.0365690534054 266790.983803
461
0.0535567795734 266602.777422
462
0.11251330438 267059.920749
463
0.0564212944909 267060.800136
464
0.143715515796 267677.736588
465
0.26124405125 266663.846701
466
0.221671798877 267149.440357
467
0.207250195319 266634.299853
468
0.154313041278 266363.996966
469
0.0859921051805 266316.927334
470
0.0554223517482 266383.11401
471
0.435766293006 268557.093866
472
0.397558670062 267592.031137
473
0.208622156685 267539.353924
474
0.255298561313 266731.452051
475
0.240321105929 266130.38966
476
0.147598170848 265984.349981
477
0.0810453473437 265945.802136
478
0.0409428054097 265948.036781
479
0.125272235914 265390.605264
480
0.0710384141469 265346.007455
481
0.0633521163058 265198.300428
482
0.0582891217137 265057.145644
483
0.0617446000551 264884.328177
484
0.0366254430738 264853.849828
48